In [1]:
from rt_utils import RTStructBuilder
import numpy as np
import os
import pandas as pd
import sys
sys.path.append('../')
from config import load_config

In [10]:
config = load_config(os.path.abspath("../config.yml"))
df_path = config.DATA_FILE
DICOM_path = config.DICOM_PATH
voxelized_save_dir = config.VOXELIZED_SAVE_PATH

all_contours = [
    config.LUNG_LEFT,
    config.LUNG_RIGHT,
    config.BODY,
    config.HEART,
    config.ESOPHAGUS,
    config.AORTA,
    config.PULMONARYBRONCHIALTREE,
    config.CHESTWALL,
    config.GREATVESSELS,
    config.SPINALCANAL,
    config.TRACHEA,
    config.VENACAVA_INF,
    config.VENACAVA_SUP
]

In [11]:
df = pd.read_csv(df_path)

for i in range(df.shape[0]):
    completeFlag = True
    patient = df.iloc[i]["Patient"]
    rt_struct_folders = []
    ptvs = df.iloc[i]["PTVs"].split(",")
    igtvs = df.iloc[i]["IGTVs"].split(",")
    print(patient)

    for p in ptvs:
        all_contours.append(p)
    for i in igtvs:
        all_contours.append(i)

    patient_path = os.path.join(DICOM_path, patient)
    dicom_series_path = os.path.join(patient_path, patient_path)

    all_folders = os.listdir(dicom_series_path)
    for a in all_folders:
        if "rts" in a.lower():
                rt_struct_folders.append(a)

    for r in rt_struct_folders:
        rt_struct_path = os.path.join(dicom_series_path, r, os.listdir(os.path.join(dicom_series_path, r))[0])

        rtstruct = RTStructBuilder.create_from(
                  dicom_series_path=dicom_series_path, 
                  rt_struct_path=rt_struct_path
                )
        existing_structs = rtstruct.get_roi_names()

        for s in all_contours:
            if s not in existing_structs and s + "~" not in existing_structs:
                print("!!!!!Missing " + s)
                completeFlag = False
        #if completeFlag:
            # print("All Structures Present")
        if not completeFlag:
            print("Missing structure(s)")

        #Convert and save
        save_dir = os.path.join(voxelized_save_dir, patient)
        os.makedirs(os.path.join(save_dir), exist_ok=True)
        for_save_dict = {}
        for name in existing_structs:
            if "iso" not in name.lower():
                try:
                    mask_3d = rtstruct.get_roi_mask_by_name(name)
                    if name[-1] == "~":
                        name = name[:-1]
                    for_save_dict[name] = mask_3d
                    
                    #np.save(os.path.join(save_dir, name + ".npy"), mask_3d)
                except:
                    print("!!!!!!!!!Error on ", name)
                    
        np.savez_compressed(os.path.join(save_dir,  "VoxelizedStructs.npz"), **for_save_dict)
        
print("end")

YONSEI_0001
end
